# Welcome! to Breast Cancer Classification 
**Greeting all !**
Hope you all are doing well


## Attribute Information:

-  ID number 
- Diagnosis (M = malignant, B = benign)

### Ten real-valued features are computed for each cell nucleus:

- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter^2 / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry
- fractal dimension ("coastline approximation" - 1)

##  Importing libraries

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

plt.style.use('ggplot')

## Load the data

In [ ]:
df = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")

In [ ]:
df.head()

## Data Preprocessing

1st we going to drop Id becouse it does not impact prediction
and Unnamed: 32 becouse it have lots of missing values

In [ ]:
df.drop(['id', 'Unnamed: 32'], axis = 1, inplace = True)

In [ ]:
df.diagnosis.unique()

Diagnosis (M = malignant, B = benign)
here we going to label encoding (M=1 & B=0)

In [ ]:
df['diagnosis'] = df['diagnosis'].apply(lambda val: 1 if val == 'M' else 0)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

**Using info() we can see three is no Nan values and all features are numerical**

In [ ]:
df.shape

In [ ]:
#counting duplicate 

df.duplicated().sum()

So here you can see three is no duplicate value found

## Exploratory Data Analysis (EDA)

In [ ]:
#relating vareiable plot

sns.relplot(x="radius_mean",y="texture_mean",hue="diagnosis",data=df);

by the above plot you can findout relationship between features and outcome

In [ ]:
#aggegation and representing uncertainty
sns.relplot(x="radius_mean",y="texture_mean",kind="line",data=df)

In [ ]:
#how does the relationship between two variable chance as fuction of a thired variable

sns.lmplot(x="radius_mean",y="texture_mean",hue="diagnosis",data=df);

In [ ]:
#here we going to plot histogram for see frequency dist
df["radius_mean"].plot(kind = 'hist',bins = 200,figsize = (6,6))
plt.title("radius_mean")
plt.xlabel("radius_mean")
plt.ylabel("Frequency")
plt.show()

## Swarm plot

In [ ]:
sns.violinplot(x="diagnosis",y="radius_mean",data=df)
sns.swarmplot(x="diagnosis",y="radius_mean",data=df)

# Pair plot

In [ ]:
#here wr checking outliers 
f,ax=plt.subplots()
sns.violinplot(data=df.iloc[:,1:3])
sns.despine(offset=10,trim=True)

In [ ]:
f,ax=plt.subplots()
sns.violinplot(data=df.iloc[:,6:8])
sns.despine(offset=10,trim=True)

In [ ]:
f,ax=plt.subplots()
sns.violinplot(data=df.iloc[:,9:11])
sns.despine(offset=10,trim=True)

In [ ]:
plt.figure(figsize = (20, 15))
plotnumber = 1

for column in df:
    if plotnumber <= 30:
        ax = plt.subplot(5, 6, plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column)
        
    plotnumber += 1

plt.tight_layout()
plt.show()

**By the above plot we can see data distribution and also identify outliers**

In [ ]:
# heatmap 

plt.figure(figsize = (20, 12))

corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

sns.heatmap(corr, mask = mask, linewidths = 1, annot = True, fmt = ".2f")
plt.show()

### We can see that there are many columns which are very highly correlated which causes multicollinearity so we have to remove highly correlated features.

In [ ]:
# removing highly correlated features

corr_matrix = df.corr().abs() 

mask = np.triu(np.ones_like(corr_matrix, dtype = bool))
tri_df = corr_matrix.mask(mask)

to_drop = [x for x in tri_df.columns if any(tri_df[x] > 0.92)]

df = df.drop(to_drop, axis = 1)

print(f"The reduced dataframe has {df.shape[1]} columns.")

# #  dealing with Outliers

In [ ]:
def mod_outlier(df):
        df1 = df.copy()
        df = df._get_numeric_data()


        q1 = df.quantile(0.25)
        q3 = df.quantile(0.75)

        iqr = q3 - q1

        lower_bound = q1 -(1.5 * iqr) 
        upper_bound = q3 +(1.5 * iqr)


        for col in df.columns:
            for i in range(0,len(df[col])):
                if df[col][i] < lower_bound[col]:            
                    df[col][i] = lower_bound[col]

                if df[col][i] > upper_bound[col]:            
                    df[col][i] = upper_bound[col]    


        for col in df.columns:
            df1[col] = df[col]

        return(df1)

df = mod_outlier(df)

In [ ]:
# creating features and label 

X = df.drop('diagnosis', axis = 1)
y = df['diagnosis']

In [ ]:
# splitting data into training and test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

**using StandardScaler to scal the data**

In [ ]:
# scaling data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression

In [ ]:
# fitting data to model

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

In [ ]:
# model predictions

y_pred = log_reg.predict(X_test)

In [ ]:
# accuracy score

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(accuracy_score(y_train, log_reg.predict(X_train)))

log_reg_acc = accuracy_score(y_test, log_reg.predict(X_test))
print(log_reg_acc)

In [ ]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

In [ ]:
# classification report

print(classification_report(y_test, y_pred))

# K Neighbors Classifier (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
# model predictions 

y_pred = knn.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, knn.predict(X_train)))

knn_acc = accuracy_score(y_test, knn.predict(X_test))
print(knn_acc)

In [ ]:
# confusion matrix

print(confusion_matrix(y_test, y_pred))

In [ ]:
# classification report

print(classification_report(y_test, y_pred))

# Support Vector Classifier (SVC)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc = SVC()
parameters = {
    'gamma' : [0.0001, 0.001, 0.01, 0.1],
    'C' : [0.01, 0.05, 0.5, 0.1, 1, 10, 15, 20]
}

grid_search = GridSearchCV(svc, parameters)
grid_search.fit(X_train, y_train)

In [ ]:
# best parameters

grid_search.best_params_

In [ ]:
svc = SVC(C = 15, gamma = 0.01)
svc.fit(X_train, y_train)

In [ ]:
# model predictions 

y_pred = svc.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, svc.predict(X_train)))

svc_acc = accuracy_score(y_test, svc.predict(X_test))
print(svc_acc)

In [ ]:
# confusion matrix

cf_matrix=confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,10))
sns.heatmap(cf_matrix,annot=True,fmt='d')

In [ ]:
# classification report

print(classification_report(y_test, y_pred))

# SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
parameters = {
    'alpha' : [0.0001, 0.001, 0.01, 0.1, 1],
    'loss' : ['hinge', 'log'],
    'penalty' : ['l1', 'l2']
}

grid_search = GridSearchCV(sgd, parameters, cv = 10, n_jobs = -1)
grid_search.fit(X_train, y_train)

In [ ]:
# best parameter 

grid_search.best_params_

In [ ]:
sgd = SGDClassifier(alpha = 0.01, loss = 'log', penalty = 'l2')
sgd.fit(X_train, y_train)

In [ ]:
# model predictions 

y_pred = sgd.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, sgd.predict(X_train)))

sgd_acc = accuracy_score(y_test, sgd.predict(X_test))
print(sgd_acc)

In [ ]:
# classification report

print(classification_report(y_test, y_pred))

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rand_clf = RandomForestClassifier(criterion = 'entropy', max_depth = 11, max_features = 'auto',
                                  min_samples_leaf = 2, min_samples_split = 3, n_estimators = 130)
rand_clf.fit(X_train, y_train)

In [ ]:
y_pred = rand_clf.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, rand_clf.predict(X_train)))

ran_clf_acc = accuracy_score(y_test, y_pred)
print(ran_clf_acc)

In [ ]:
# classification report

print(classification_report(y_test, y_pred))

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()

parameters = {
    'loss': ['deviance', 'exponential'],
    'learning_rate': [0.001, 0.1, 1, 10],
    'n_estimators': [100, 150, 180, 200]
}

grid_search_gbc = GridSearchCV(gbc, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search_gbc.fit(X_train, y_train)

In [ ]:
# best parameters 

grid_search_gbc.best_params_

In [ ]:
gbc = GradientBoostingClassifier(learning_rate = 1, loss = 'exponential', n_estimators = 180)
gbc.fit(X_train, y_train)

In [ ]:
y_pred = gbc.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, gbc.predict(X_train)))

gbc_acc = accuracy_score(y_test, y_pred)
print(gbc_acc)

In [ ]:
# classification report

print(classification_report(y_test, y_pred))

# Extreme Gradient Boosting

In [ ]:
from xgboost import XGBClassifier 

xgb = XGBClassifier(objective = 'binary:logistic', learning_rate = 0.5, max_depth = 5, n_estimators = 180)

xgb.fit(X_train, y_train)

In [ ]:
y_pred = xgb.predict(X_test)

In [ ]:
# accuracy score

print(accuracy_score(y_train, xgb.predict(X_train)))

xgb_acc = accuracy_score(y_test, y_pred)
print(xgb_acc)

In [ ]:
# classification report

print(classification_report(y_test, y_pred))

In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'SVC', 'SGD Classifier', 'Random Forest Classifier', 
            'Gradient Boosting Classifier','XgBoost'],
    'Score': [log_reg_acc, knn_acc, svc_acc, sgd_acc,ran_clf_acc, gbc_acc, xgb_acc]
})

models.sort_values(by = 'Score', ascending = False)

### Best model for diagnosing breast cancer is "Support Vector Classifier" with an accuracy of 98.9%.

## If you like my work, please do a upvote.

**Thank you**